In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
data_set=pd.read_csv("../input/delhi-weather-data/testset.csv")
data_set.head(5)

In [3]:
data_set.columns=data_set.columns.str.replace("_","")
data_set.columns=data_set.columns.str.replace(" ","")

In [4]:
data_set.columns

In [5]:
data_set=data_set[["datetimeutc","tempm"]]
data_set.head(3)

In [6]:
data_set.isna().sum()

In [7]:
data_set.fillna(data_set.mean(),inplace=True)

In [8]:
data_set["datetimeutc"]=pd.to_datetime(data_set["datetimeutc"])

In [9]:
data_set.head(3)

In [10]:
data_set.index=data_set["datetimeutc"]
data_set.drop(["datetimeutc"],axis=1,inplace=True)
data_set.head(3)

In [11]:
plt.figure(figsize=(20,7))
plt.plot(data_set["tempm"])
plt.show()

In [41]:
data=data_set.values

In [42]:
data.shape

In [43]:
train_size=len(data)-10
test_size=10

In [44]:
scaler=MinMaxScaler()
scaler_data=scaler.fit_transform(data)

In [60]:
x_train,x_test=[],[]

for i in range(30,train_size):
    x_train.append(scaler_data[i-30:i,0])
    x_test.append(scaler_data[i,0])

x_train,x_test=np.array(x_train),np.array(x_test)

In [61]:
x_train.shape

In [62]:
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))

In [63]:
x_train.shape

In [64]:
x_test.shape

In [21]:
from tensorflow.keras import layers

In [65]:
model1=Sequential()
model1.add(layers.Conv1D(filters=128, kernel_size=2, activation="relu", input_shape=(30,1)))
model1.add(layers.Conv1D(filters=128, kernel_size=2, activation="relu"))
model1.add(layers.MaxPooling1D(pool_size=2))
model1.add(layers.Conv1D(filters=256, kernel_size=2, activation="relu"))
model1.add(layers.Flatten())
model1.add(layers.RepeatVector(x_train.shape[1])) #same as input (30,1)
model1.add(layers.LSTM(units=100, return_sequences=True, activation="relu"))
model1.add(layers.Dropout(0.2))
model1.add(layers.LSTM(units=100, return_sequences=True, activation="relu"))
model1.add(layers.Dropout(0.2))
model1.add(layers.Bidirectional(layers.LSTM(units=128, activation="relu")))
model1.add(layers.Dense(100, activation="relu"))
model1.add(layers.Dense(1))

In [66]:
model1.compile(optimizer="adam",loss="mean_squared_error")
model1.fit(x_train,x_test,epochs=4)

In [68]:
test=data_set[len(data_set)-test_size-30:].values
test=scaler.transform(test)

In [69]:
x_test=[]

for i in range(30,len(test)):
    x_test.append(test[i-30:i,0])

x_test=np.array(x_test)

In [70]:
x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))

In [71]:
predict=scaler.inverse_transform(model1.predict(x_test))

In [72]:
valid=data_set[train_size:]
valid["Predicted"]=predict

In [73]:
valid

In [74]:
from sklearn.metrics import mean_squared_error

mean_squared_error(valid["tempm"].values,valid["Predicted"].values)

In [75]:
model1.save("weather_prediction_model.h5")

In [77]:
plt.figure(figsize=(20,10))
plt.plot(valid[["tempm","Predicted"]])
plt.show()